<a href="https://colab.research.google.com/github/nishantdotcom/Control-and-protection-of-HVDC-AC-electrical-grids-IEEE-IES-MOOC/blob/main/Kaggle_leather_surface_defect_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
praveen2084_leather_defect_classification_path = kagglehub.dataset_download('praveen2084/leather-defect-classification')

print('Data source import complete.')


Using Colab cache for faster access to the 'leather-defect-classification' dataset.
Data source import complete.


In [5]:
print(praveen2084_leather_defect_classification_path)

/kaggle/input/leather-defect-classification


In [6]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
train_generator = train_datagen.flow_from_directory(
    '/kaggle/input/leather-defect-classification/Leather Defect Classification',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',  # or 'binary' if you have two classes
    subset='training'  # set as training data
)

Found 2880 images belonging to 6 classes.


In [9]:
validation_generator = train_datagen.flow_from_directory(
    '/kaggle/input/leather-defect-classification/Leather Defect Classification',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',  # or 'binary' if you have two classes
    subset='validation'  # set as validation data
)

Found 720 images belonging to 6 classes.


In [10]:
test_generator = test_datagen.flow_from_directory(
    '/kaggle/input/leather-defect-classification/Leather Defect Classification',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'  # or 'binary' if you have two classes
)

Found 3600 images belonging to 6 classes.


In [11]:
def create_cnn_model(input_shape, num_classes):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    return model

In [12]:
def create_resnet_model(input_shape, num_classes):
    base_model = tf.keras.applications.ResNet50(input_shape=input_shape, include_top=False, weights='imagenet')
    base_model.trainable = False

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

In [13]:
def create_vgg_model(input_shape, num_classes):
    model = models.Sequential()
    model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    return model

In [14]:
input_shape = (224, 224, 3)

In [15]:
num_classes = 6

In [16]:
cnn_model = create_cnn_model(input_shape, num_classes)
resnet_model = create_resnet_model(input_shape, num_classes)
vgg_model = create_vgg_model(input_shape, num_classes)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [17]:
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
resnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
vgg_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history_cnn = cnn_model.fit(train_generator, epochs=10, validation_data=validation_generator)
history_resnet = resnet_model.fit(train_generator, epochs=10, validation_data=validation_generator)
history_vgg = vgg_model.fit(train_generator, epochs=10, validation_data=validation_generator)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 253s 3s/step - accuracy: 0.3580 - loss: 2.6998 - val_accuracy: 0.4736 - val_loss: 1.1413
Epoch 2/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 245s 3s/step - accuracy: 0.6124 - loss: 0.8986 - val_accuracy: 0.6597 - val_loss: 1.0980
Epoch 3/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 249s 3s/step - accuracy: 0.7530 - loss: 0.5850 - val_accuracy: 0.6778 - val_loss: 1.0728
Epoch 4/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 240s 3s/step - accuracy: 0.8014 - loss: 0.4714 - val_accuracy: 0.6000 - val_loss: 1.5313
Epoch 5/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 241s 3s/step - accuracy: 0.8042 - loss: 0.4742 - val_accuracy: 0.6667 - val_loss: 0.9963
Epoch 6/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 250s 3s/step - accuracy: 0.8067 - loss: 0.4605 - val_accuracy: 0.6014 - val_loss: 1.1190
Epoch 7/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 242s 3s/step - accuracy: 0.8225 - loss: 0.4214 - val_accuracy: 0.6583 - val_loss: 1.0336
Epoch 8/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 249s 3s/step - accuracy: 0.8879 - loss: 0.3092 - val_accuracy: 0.6681 - v

In [ ]:
plt.plot(history_cnn.history['accuracy'], label='CNN Training Accuracy')
plt.plot(history_resnet.history['accuracy'], label='ResNet Training Accuracy')
plt.plot(history_vgg.history['accuracy'], label='VGG Training Accuracy')

plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Model Comparison - Training Accuracy')
plt.legend()
plt.show()

In [ ]:
# Print accuracy for CNN model
cnn_accuracy = history_cnn.history['accuracy'][-1]
cnn_val_accuracy = history_cnn.history['val_accuracy'][-1]
print(f"CNN Model - Training Accuracy: {cnn_accuracy:.2f}, Validation Accuracy: {cnn_val_accuracy:.2f}")

# Print accuracy for ResNet model
resnet_accuracy = history_resnet.history['accuracy'][-1]
resnet_val_accuracy = history_resnet.history['val_accuracy'][-1]
print(f"ResNet Model - Training Accuracy: {resnet_accuracy:.2f}, Validation Accuracy: {resnet_val_accuracy:.2f}")

# Print accuracy for VGG model
vgg_accuracy = history_vgg.history['accuracy'][-1]
vgg_val_accuracy = history_vgg.history['val_accuracy'][-1]
print(f"VGG Model - Training Accuracy: {vgg_accuracy:.2f}, Validation Accuracy: {vgg_val_accuracy:.2f}")

In [ ]:
def create_resnet_model(input_shape, num_classes):
    base_model = tf.keras.applications.ResNet50(input_shape=input_shape, include_top=False, weights='imagenet')

    # Unfreeze the last few layers for fine-tuning
    for layer in base_model.layers[-10:]:
        layer.trainable = True

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

In [ ]:
resnet_model = create_resnet_model(input_shape, num_classes)

In [ ]:
resnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history_resnet = resnet_model.fit(train_generator, epochs=20, validation_data=validation_generator)

In [ ]:
plt.plot(history_resnet.history['accuracy'], label='ResNet Training Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training Accuracy')
plt.legend()
plt.show()

In [ ]:
resnet_accuracy = history_resnet.history['accuracy'][-1]
resnet_val_accuracy = history_resnet.history['val_accuracy'][-1]
print(f"ResNet Model - Training Accuracy: {resnet_accuracy:.2f}, Validation Accuracy: {resnet_val_accuracy:.2f}")

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)
callbacks = [early_stopping, lr_scheduler]

In [ ]:
resnet_model_1 = create_resnet_model(input_shape, num_classes)

In [ ]:
resnet_model_1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history_resnet_1 = resnet_model_1.fit(train_generator, epochs=20, validation_data=validation_generator, callbacks=callbacks)

In [ ]:
plt.plot(history_resnet.history['accuracy'], label='ResNet Training Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training Accuracy')
plt.legend()
plt.show()

In [ ]:
resnet_accuracy = history_resnet.history['accuracy'][-1]
resnet_val_accuracy = history_resnet.history['val_accuracy'][-1]
print(f"ResNet Model - Training Accuracy: {resnet_accuracy:.2f}, Validation Accuracy: {resnet_val_accuracy:.2f}")